# 🍎 Smart Fitness & Diet AI Agent — Google Colab Version
### Zero Errors • Zero Conflicts • No Version Pins
Simply run all cells top to bottom.


In [ ]:
# 📦 Install dependencies (NO VERSION NUMBERS)
!pip install -q gradio sentence-transformers pillow pandas

In [ ]:
import io, time, json, uuid
import numpy as np
import pandas as pd
from PIL import Image
import gradio as gr
from sentence_transformers import SentenceTransformer

print('Imports loaded successfully!')

## 🔹 Nutrition Dataset (RAG Memory Source)


In [ ]:
nutrition_df = pd.DataFrame([
    {"food_id": "chicken_salad", "name": "Chicken Salad", "calories": 420, "protein": 30, "fat": 15, "carbs": 20},
    {"food_id": "grilled_fish", "name": "Grilled Fish", "calories": 350, "protein": 40, "fat": 8, "carbs": 5},
    {"food_id": "oatmeal", "name": "Oatmeal", "calories": 300, "protein": 10, "fat": 6, "carbs": 52},
    {"food_id": "veggie_salad", "name": "Veggie Salad", "calories": 180, "protein": 5, "fat": 7, "carbs": 20}
])
nutrition_df

## 🔹 Observability


In [ ]:
LOGS = []
METRICS = {"vision":0, "rag":0, "plan":0, "judge":0}

def log(agent, action, detail):
    LOGS.append({"ts":time.time(), "agent":agent, "action":action, "detail":detail})

def metric(name):
    METRICS[name] += 1

def get_logs(): return LOGS.copy()
def get_metrics(): return METRICS.copy()

## 🔹 MCP-Style Long-Running Operations


In [ ]:
PENDING_OPS = {}

def start_lro(session_id, payload):
    op_id = str(uuid.uuid4())
    PENDING_OPS[op_id] = {"session_id": session_id, "payload": payload, "status": "pending"}
    return op_id

def resume_lro(op_id):
    if op_id not in PENDING_OPS:
        return {"error":"invalid"}
    PENDING_OPS[op_id]["status"] = "resolved"
    return PENDING_OPS[op_id]

## 🔹 Vector Memory (Pure NumPy)


In [ ]:
class VectorStore:
    def __init__(self):
        self.model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
        self.docs = []

    def add(self, text, metadata=None):
        emb = self.model.encode(text)
        self.docs.append({"text":text, "emb":emb, "metadata":metadata})

    def query(self, q, k=3):
        qv = self.model.encode(q)
        sims = [np.dot(qv, d['emb']) for d in self.docs]
        idxs = np.argsort(sims)[::-1][:k]
        return [self.docs[i] for i in idxs]

## 🔹 Session + Load Memory


In [ ]:
memory = VectorStore()
for _, row in nutrition_df.iterrows():
    txt = f"{row['name']} {row['calories']} calories protein {row['protein']} fat {row['fat']} carbs {row['carbs']}"
    memory.add(txt, metadata=row.to_dict())
len(memory.docs)

## 🔹 Agents (Vision, RAG, Planner, Judge)


In [ ]:
def identify_food(image_bytes):
    metric("vision")
    return {"items":[{"food_id":"chicken_salad","label":"Chicken Salad","confidence":0.93}]}

def lookup_nutrition(fid):
    metric("rag")
    return memory.query("chicken salad",1)[0]["metadata"]

def generate_plan(profile, seed):
    metric("plan")
    return {"daily_calories":profile.get("target_calories",2000),"days":[{"day":1,"meal":seed["label"]}]}

def judge_plan(plan, profile):
    metric("judge")
    return {"score":90}

## 🔹 Orchestrator


In [ ]:
def orchestrator(image_bytes, profile):
    vis = identify_food(image_bytes)
    food = vis['items'][0]
    nutri = lookup_nutrition(food['food_id'])
    plan = generate_plan(profile, food)
    score = judge_plan(plan, profile)
    return {"status":"done", "plan":plan, "nutrition":nutri}

## 🔹 Gradio UI


In [ ]:
def ui_process(image, profile_json):
    profile = json.loads(profile_json)
    buf = io.BytesIO()
    image.save(buf, format='PNG')
    return orchestrator(buf.getvalue(), profile)

with gr.Blocks() as demo:
    gr.Markdown('# 🍎 Smart Fitness & Diet Agent')
    img = gr.Image(type='pil')
    prof = gr.Textbox(value=json.dumps({"target_calories":2000}))
    btn = gr.Button("Run Agent")
    out = gr.JSON()
    btn.click(ui_process, [img, prof], out)

demo.launch()